In [2]:
import pandas as pd
import numpy as np
import pickle
import Bio   
import plotly.express as px
import plotly.graph_objects as go
from jproperties import Properties
#import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModel, TrainingArguments, Trainer
from motif_utils import seq2kmer # Soruced from https://github.com/jerryji1993/DNABERT
import torch
from datasets import Dataset
import evaluate
import json
from load_data import create_dataset, explode_dna
import wandb

from transformers.onnx import FeaturesManager
import transformers
from pathlib import Path
from sklearn.preprocessing import StandardScaler

c:\Users\zeusg\Envs\real-fast\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
with open("./data/gene_symbol_protein_sequences.pkl", 'rb') as file:
        protein =  pickle.load(file)

In [4]:
protein

,gene_symbol_harmonized,db,UniqueIdentifier,EntryName,ProteinName,OS,OX,GN,PE,SV,motifs,seq
0,Hsf1,tr,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor protein 1,Mus musculus,10090,Hsf1,1,1,NaN,MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...
1,Auts2,sp,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Mus musculus,10090,Auts2,1,2,NaN,MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...
2,Ptgs2,tr,A0A087WPT2,A0A087WPT2_MOUSE,Prostaglandin G/H synthase 2 (Fragment),Mus musculus,10090,Ptgs2,4,1,NaN,MSTGFDQYKCDCTRTGFYGENCTTPEFLTRIKLLLKPTPNTVHYIL...
3,Fat1,tr,A0A087WPU4,A0A087WPU4_MOUSE,FAT atypical cadherin 1 (Fragment),Mus musculus,10090,Fat1,1,1,NaN,XRPYFDSKLNKNIYSDIPPQVPVRPISYTPSIPSDSRNNLDRNSFE...
4,Gm20905,tr,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814",Mus musculus,10090,Gm20905,3,1,NaN,MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...
...,...,...,...,...,...,...,...,...,...,...,...,...
94781,Snph,tr,Z4YN82,Z4YN82_MOUSE,Syntaphilin (Fragment),Mus musculus,10090,Snph,1,1,NaN,MAMSLQGSRRASAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSCKGS...
94782,Timm22,tr,Z4YN86,Z4YN86_MOUSE,Mitochondrial import inner membrane translocas...,Mus musculus,10090,Timm22,1,1,NaN,MAATAPKAGGSAPEAAGSAEAPLQYSLLLQYLVGDKRQPRLLEPGS...
94783,Akap17b,tr,Z4YN92,Z4YN92_MOUSE,A-kinase anchor protein 17B (Fragment),Mus musculus,10090,Akap17b,1,1,NaN,MFDTTKHFSEGAIQRRNQERLKLQELEEERKKEKKREEEVAERKRK...
94784,Mau2,tr,Z4YNA3,Z4YNA3_MOUSE,MAU2 chromatid cohesion factor homolog,Mus musculus,10090,Mau2,1,1,NaN,MAAQAAAAAQAAAAAQAAAAQAAQAEAAESWYLALLGFAEHFRTSS...


In [10]:
protein.describe()

,db,UniqueIdentifier,EntryName,ProteinName,OS,OX,GN,PE,SV,seq
count,94786,94786,94786,94786,94786,94786,91439,86453,86453,94786
unique,2,94786,86453,52466,1,1,25638,5,10,91450
top,tr,A0A075F5C6,PCD15_MOUSE,Olfactory receptor,Mus musculus,10090,H2-K1,1,1,WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...
freq,69316,1,26,1561,94786,94786,312,38890,75796,12


In [14]:
protein.seq.str.len()
fig = go.Figure()
fig.add_trace(go.Histogram(x = protein.seq.str.len()))
fig.show()

In [7]:
print(len(protein[protein.seq.str.len() <= 512])/len(protein))
protein = protein[protein.seq.str.len() <= 512]

0.7397611461608254


In [11]:
protein.seq[2]

'MSTGFDQYKCDCTRTGFYGENCTTPEFLTRIKLLLKPTPNTVHYILTHFKGVWNIVNNIPFL'

In [12]:
protein

,gene_symbol_harmonized,db,UniqueIdentifier,EntryName,ProteinName,OS,OX,GN,PE,SV,motifs,seq
2,Ptgs2,tr,A0A087WPT2,A0A087WPT2_MOUSE,Prostaglandin G/H synthase 2 (Fragment),Mus musculus,10090,Ptgs2,4,1,NaN,MSTGFDQYKCDCTRTGFYGENCTTPEFLTRIKLLLKPTPNTVHYIL...
3,Fat1,tr,A0A087WPU4,A0A087WPU4_MOUSE,FAT atypical cadherin 1 (Fragment),Mus musculus,10090,Fat1,1,1,NaN,XRPYFDSKLNKNIYSDIPPQVPVRPISYTPSIPSDSRNNLDRNSFE...
4,Gm20905,tr,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814",Mus musculus,10090,Gm20905,3,1,NaN,MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...
10,Tgfb2,tr,A0A0A6YXU9,A0A0A6YXU9_MOUSE,Transforming growth factor beta,Mus musculus,10090,Tgfb2,1,1,NaN,MHYCVLSTFLLLHLVPVALSLSTCSTLDMDQFMRKRIEAIRGQILS...
13,Arrdc4,sp,A0A0B4J1F4,ARRD4_MOUSE,Arrestin domain-containing protein 4,Mus musculus,10090,Arrdc4,1,1,PPNY PPLY,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
...,...,...,...,...,...,...,...,...,...,...,...,...
94780,Sanbr,tr,Z4YN77,Z4YN77_MOUSE,SANT and BTB domain regulator of class switch ...,Mus musculus,10090,Sanbr,4,1,NaN,MSRGYPENNNFLNNNNQMVLDMILYPLIGIPQTINWETVARLVPGLTP
94781,Snph,tr,Z4YN82,Z4YN82_MOUSE,Syntaphilin (Fragment),Mus musculus,10090,Snph,1,1,NaN,MAMSLQGSRRASAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSCKGS...
94782,Timm22,tr,Z4YN86,Z4YN86_MOUSE,Mitochondrial import inner membrane translocas...,Mus musculus,10090,Timm22,1,1,NaN,MAATAPKAGGSAPEAAGSAEAPLQYSLLLQYLVGDKRQPRLLEPGS...
94783,Akap17b,tr,Z4YN92,Z4YN92_MOUSE,A-kinase anchor protein 17B (Fragment),Mus musculus,10090,Akap17b,1,1,NaN,MFDTTKHFSEGAIQRRNQERLKLQELEEERKKEKKREEEVAERKRK...
